In [15]:
def find_min_max_positions_bim(bim_file):
    with open(bim_file, 'r') as f:
        positions = [int(line.split()[3]) for line in f]
    
    min_pos = min(positions)
    max_pos = max(positions)
    
    return min_pos, max_pos

bim_file = "data/fourier_ls-chr6-1167_train.bim"
idx1, idx2 = find_min_max_positions_bim(bim_file)

print(f"Min position: {idx1}")
print(f"Max position: {idx2}")

Min position: 30798252
Max position: 31568419


In [2]:
import subprocess
import os
train_bcf = 'data/fourier_ls-chr6-1167_train.bcf'

result = subprocess.run([
        '/scratch2/prateek/impute5_v1.2.0/xcftools_static', 'view',
        '-i', train_bcf,
        '-o', f"train_xcf.bcf",
        '-O', 'sh',
        '-r', '6',
        '-T8',
        '-m', '0.03125'
    ])

train_xcf = "train_xcf.bcf"


[XCFtools] Convert from/to XCF files
  * Authors       : Olivier DELANEAU, University of Lausanne
  * Contact       : olivier.delaneau@gmail.com
  * Version       : 1.0.0 / commit = 48abb98 / release = 2023-07-25
  * Run date      : 23/10/2024 - 10:45:47

Files:
  * Input BCF     : [fourier_ls-chr6-1167_train.bcf]
  * Output XCF    : [train_xcf.bcf]

Parameters:
  * Seed    : 15052011
  * Threads : 8 threads
  * MAF     : 0.03125

Converting from BCF to XCF [Sparse/Haplotype]
  * Region        : 6
  * Contig        : 6
  * Min MAF       : 0.03125
  * #samples = 225245
  * Number of BCF records processed: Nc=0/ Nr=0

Finalization:
  * Total running time = 0 seconds


In [106]:
import numpy as np

def process_plink_data(plink_file_prefix):
    vcf_file = f"data/{plink_file_prefix}.vcf"
    # print("Exporting PLINK data to VCF...")
    export_command = [
        "../plink2", "--bfile", plink_file_prefix,
        "--recode", "vcf", "--out", plink_file_prefix,
        "--silent"
    ]
    
    if not os.path.exists(vcf_file):
        subprocess.run(export_command, check=True)

    bcftools_command = [
        "../bcftools/bcftools", "view", "-Oz", "-o", f"data/{plink_file_prefix}.bcf", vcf_file
    ]

    if not os.path.exists(f"data/{plink_file_prefix}.bcf"):
        subprocess.run(bcftools_command, check=True)

        # print("BCF conversion complete.")

        # print("Adding AC field..")
        subprocess.run([
        '../bcftools/bcftools', '+fill-tags', f'data/{plink_file_prefix}.bcf', '-Ob', '-o', f'data/{plink_file_prefix}_AC.bcf', '--', '-t', 'AN,AC'
        ], stdout=subprocess.DEVNULL)
        # print("Adding index file..")
        subprocess.run(['../bcftools/bcftools', 'index', f'data/{plink_file_prefix}_AC.bcf'])

    if not os.path.exists(f"{plink_file_prefix}_xcf.bcf"):
        subprocess.run([
            '../impute5_v1.2.0/xcftools_static', 'view',
            '-i', f"data/{plink_file_prefix}_AC.bcf",
            '-o', f"data/{plink_file_prefix}_xcf.bcf",
            '-O', 'sh',
            '-r', '6',
            '-T8',
            '-m', '0.03125'
        ])

def process_plink_data_with_drop(plink_file_prefix, rs_ids):
    vcf_file = f"data/{plink_file_prefix}.vcf"
    # print("Exporting PLINK data to VCF...")
    export_command = [
        "../plink2", "--bfile", f"data/{plink_file_prefix}",
        "--recode", "vcf", "--out", f"data/{plink_file_prefix}",
        "--silent"
    ]
    subprocess.run(export_command)

    modified_vcf_file = f"modified_test.vcf"
    with open(vcf_file, 'r') as f:
        lines = f.readlines()

    with open(modified_vcf_file, 'w') as out_file:
        for line in lines:
            if line.startswith('#'):
                out_file.write(line)
                continue
            
            fields = line.strip().split('\t')
            snp_id = fields[2]

            if snp_id in rs_ids:
                # fields[9:] = ['./.' for _ in fields[9:]]
                continue
                
            out_file.write('\t'.join(fields) + '\n')

    # print("VCF modification complete.")

    bcftools_command = [
        "../bcftools/bcftools", "view", "-Oz", '--threads', '32', "-o", "modified_test.bcf", modified_vcf_file
    ]
    subprocess.run(bcftools_command)

    # print("BCF conversion complete.")

    # print("Adding AC field..")
    subprocess.run([
     '../bcftools/bcftools', '+fill-tags', 'modified_test.bcf', '-Ob', '-o', 'modified_test_AC.bcf', '--', '-t', 'AN,AC'
    ], stdout=subprocess.DEVNULL)
    # print("Adding index file..")
    subprocess.run(['../bcftools/bcftools', 'index', 'modified_test_AC.bcf'])

    # print('Converting to XCF...')
    subprocess.run([
        '../impute5_v1.2.0/xcftools_static', 'view',
        '-i', 'modified_test_AC.bcf',
        '-o', "modified_test_xcf.bcf",
        '-O', 'sh',
        '-r', '6',
        '-T8',
        '-m', '0.03125'
    ])

def extract_imputed_genotype_array(vcf_file, target_snp):

    with open(vcf_file, 'r') as file:
        lines = file.readlines()
        
        for line in lines:
            if line.startswith('#'):
                continue  # Skip header lines
            
            fields = line.strip().split('\t')
            snp_id = f"{fields[0]}:{fields[1]}"  # Format: chr:pos
            
            if snp_id == target_snp:
                genotype_data = fields[9:]  # Genotype fields for all samples
                genotype_array = []

                # Parse genotype data
                for genotype in genotype_data:
                    gt_info = genotype.split(':')[0]  # Take the genotype part (GT)
                    # Convert genotype format to 0, 1, or 2
                    if gt_info == "0|0":
                        genotype_array.append(0)
                    elif gt_info == "0|1" or gt_info == "1/0":
                        genotype_array.append(1)
                    elif gt_info == "1|1":
                        genotype_array.append(2)
                    else:
                        genotype_array.append(-1)  # for missing or unknown genotypes
                
                return genotype_array
    
    return None  # SNP not found in the VCF file


def extract_test_genotype_array(vcf_file, target_snp):

    with open(vcf_file, 'r') as file:
        lines = file.readlines()
        
        for line in lines:
            if line.startswith('#'):
                continue  # Skip header lines
            
            fields = line.strip().split('\t')
            snp_id = f"{fields[0]}:{fields[1]}"  # Format: chr:pos
            
            if snp_id == target_snp:
                genotype_data = fields[9:]  # Genotype fields for all samples
                genotype_array = []

                # Parse genotype data
                for genotype in genotype_data:
                    # Convert genotype format to 0, 1, or 2
                    if genotype == "0/0":
                        genotype_array.append(0)
                    elif genotype == "0/1" or genotype == "1/0":
                        genotype_array.append(1)
                    elif genotype == "1/1":
                        genotype_array.append(2)
                    else:
                        genotype_array.append(-1)  # for missing or unknown genotypes
                
                return genotype_array
    
    return None  # SNP not found in the VCF file

def compute_r_squared(array1, array2):
    # Ensure both arrays are NumPy arrays
    array1 = np.array(array1)
    array2 = np.array(array2)

    # Calculate the correlation coefficient
    correlation_matrix = np.corrcoef(array1, array2)
    correlation_xy = correlation_matrix[0, 1]
    
    # Calculate R^2
    r_squared = correlation_xy ** 2
    
    return r_squared

In [107]:
buffer_region = f"6:{idx1}-{idx2}"

os.environ['BCFTOOLS_PLUGINS'] = '/scratch2/prateek/bcftools/plugins'

plink_file_train_prefix = "fourier_ls-chr6-1167_train"
plink_file_test_prefix = "fourier_ls-chr6-1167_test"

process_plink_data(plink_file_train_prefix)

bim_file = f"{plink_file_test_prefix}.bim"
rs_ids = []

with open(bim_file, 'r') as f:
    for line in f:
        fields = line.split()
        rs_id = fields[1]
        rs_ids.append(rs_id)

r2s = []
for idx, rs_id in enumerate(rs_ids):
    print(f"Dropping SNP: {rs_id}")
    num = int(rs_id.split(':')[1])

    process_plink_data_with_drop(plink_file_test_prefix, rs_id)
    
    result = subprocess.run([
        '../impute5_v1.2.0/impute5_v1.2.0_static',
        '--h', f'data/{plink_file_train_prefix}_xcf.bcf',
        '--m', '../b37_recombination_maps/chr6.b37.gmap.gz',
        '--g', 'modified_test_xcf.bcf',
        '--r', f"6:{num}-{num+1}",
        '--buffer-region', buffer_region,
        '--o', f"imputed_{idx}.vcf",
        '--l', f"imputed_{idx}.log",
        '--threads', '32'
    ])
    
    imputed_genotype_array = extract_imputed_genotype_array(f'imputed_{idx}.vcf', rs_id)
    test_genotype_array = extract_test_genotype_array('data/fourier_ls-chr6-1167_test.vcf', rs_id)

    r2 = compute_r_squared(imputed_genotype_array, test_genotype_array)
    print(r2)
    r2s.append(r2)
    
    # Delete the generated files after the SNP is processed
    files_to_delete = [f"imputed_{idx}.log", f"imputed_{idx}.vcf", "modified_test_AC.bcf", "modified_test_xcf.vcf", "modified_test_xcf.bcf.csi", "modified_test_xcf.bin", "modified_test_xcf.fam", "modified_test.bcf", "modified_test.vcf"]
    for file in files_to_delete:
        if os.path.exists(file):
            os.remove(file)
        else:
            print(f"File {file} not found for deletion")

Dropping SNP: 6:30798252
PLINK v2.0.0-a.6LM AVX2 AMD (20 Oct 2024)          cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to fourier_ls-chr6-1167_test.log.
Options in effect:
  --bfile fourier_ls-chr6-1167_test
  --export vcf
  --out fourier_ls-chr6-1167_test

Start time: Wed Oct 23 15:29:11 2024
128674 MiB RAM detected, ~113002 available; reserving 64337 MiB for main
workspace.
Using up to 32 threads (change this with --threads).
56312 samples (30180 females, 26132 males; 56312 founders) loaded from
fourier_ls-chr6-1167_test.fam.
1167 variants loaded from fourier_ls-chr6-1167_test.bim.
Note: No phenotype data present.
--export vcf to fourier_ls-chr6-1167_test.vcf ... 10101111121213131414151516161717181819192020212122222323242425252626272728282929303031313232333334343535363637373838393940404141424243434444454546464747484849495050515152525353545455555656575758585959606061616262636364646565666667676868696970707171727273

KeyboardInterrupt: 

In [103]:
print(r2s)

[0.4888814999377982]
